# Model deployment using Sagemaker

## 1. Inicialization

In [7]:
import boto3
import sagemaker
import os
import numpy as np

In [8]:
# session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# S3 location
bucket = sagemaker_session.default_bucket()

# Set a prefix  
prefix = 'sagemaker/disaster_tweets_classifier'

In [14]:
data_dir = "output"

data_prefix = 'sagemaker/disaster_tweets_classifier/data'

input_data = sagemaker_session.upload_data(data_dir, bucket=bucket, key_prefix=prefix)

## 2. Load and deploy saved model

TensorFlow Serving Model https://sagemaker.readthedocs.io/en/stable/sagemaker.tensorflow.html

In [9]:
#Tar the entire directory and upload to Amazon S3
import tarfile
with tarfile.open('model.tar.gz', mode='w:gz') as archive:
    archive.add('export', recursive=True)

inputs = sagemaker_session.upload_data(path='model.tar.gz', key_prefix=prefix)

In [12]:
# Specification of framework_version='2.0.0' alighns the TensorFlow version of 
# the trained model with the version of the serving model. Otherwise, an error was thrown.

from sagemaker.tensorflow.serving import Model
model = Model(model_data = 's3://' + bucket + '/' + prefix + '/' + 'model.tar.gz', 
              role = role,
             framework_version='2.0.0')

The class sagemaker.tensorflow.serving.Model has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [13]:
%%time
predictor = model.deploy(initial_instance_count=1, 
                         instance_type='ml.m4.xlarge')

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


-------------------------------------------------*

UnexpectedStatusException: Error hosting endpoint tensorflow-inference-2021-03-21-19-31-42-430: Failed. Reason:  The primary container for production variant AllTraffic did not pass the ping health check. Please check CloudWatch logs for this endpoint..

In [ ]:
predictor.endpoint

In [ ]:
data = [1,,,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1]

In [ ]:
from sagemaker.tensorflow.serving import Predictor
# create a Predictor with JSON serialization
predictor = Predictor(predictor.endpoint, serializer=sagemaker.predictor.csv_serializer)

# CSV-formatted string input
input = data

result = predictor.predict(input)
result

## 3. Delete resources

In [15]:
# Accepts a predictor endpoint as input
# And deletes the endpoint by name
def delete_endpoint(predictor):
        try:
            boto3.client('sagemaker').delete_endpoint(EndpointName=predictor.endpoint)
            print('Deleted {}'.format(predictor.endpoint))
        except:
            print('Already deleted: {}'.format(predictor.endpoint))

In [16]:
delete_endpoint(predictor)

NameError: name 'predictor' is not defined